<img src="https://nlp.johnsnowlabs.com/assets/images/logo.png" width="180" height="50" style="float: left;">

## Rule-based Sentiment Analysis

In the following example, we walk-through a simple use case for our straight forward SentimentDetector annotator.

This annotator will work on top of a list of labeled sentences which can have any of the following features
    
    positive
    negative
    revert
    increment
    decrement

Each of these sentences will be used for giving a score to text 

#### 1. Call necessary imports and set the resource path to read local data files

In [1]:
#Imports
import sys
sys.path.append('../../')

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import RegexRule
from sparknlp.base import DocumentAssembler, Finisher
#Setting location of resource Directory
resource_path= "../../../src/test/resources/"


#### 2. Load SparkSession if not already there

In [3]:
spark = SparkSession.builder \
    .appName("SentimentDetector")\
    .master("local[*]")\
    .config("spark.driver.memory","8G")\
    .config("spark.driver.maxResultSize", "2G")\
    .config("spark.jar", "lib/sparknlp.jar")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

In [4]:
data = spark. \
        read. \
        parquet(resource_path+"sentiment.parquet"). \
        limit(10000).cache()

data.show()

+------+---------+--------------------+
|itemid|sentiment|                text|
+------+---------+--------------------+
|     1|        0|                 ...|
|     2|        0|                 ...|
|     3|        1|              omg...|
|     4|        0|          .. Omga...|
|     5|        0|         i think ...|
|     6|        0|         or i jus...|
|     7|        1|       Juuuuuuuuu...|
|     8|        0|       Sunny Agai...|
|     9|        1|      handed in m...|
|    10|        1|      hmmmm.... i...|
|    11|        0|      I must thin...|
|    12|        1|      thanks to a...|
|    13|        0|      this weeken...|
|    14|        0|     jb isnt show...|
|    15|        0|     ok thats it ...|
|    16|        0|    &lt;-------- ...|
|    17|        0|    awhhe man.......|
|    18|        1|    Feeling stran...|
|    19|        0|    HUGE roll of ...|
|    20|        0|    I just cut my...|
+------+---------+--------------------+
only showing top 20 rows



#### 3. Create appropriate annotators. We are using Sentence Detection, Tokenizing the sentences, and find the lemmas of those tokens. The Finisher will only output the Sentiment.

In [8]:
document_assembler = DocumentAssembler() \
    .setInputCol("text")

sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

lemmatizer = Lemmatizer() \
    .setInputCols(["token"]) \
    .setOutputCol("lemma") \
    .setDictionary(resource_path+"lemma-corpus-small/lemmas_small.txt", key_delimiter="->", value_delimiter="\t")
        
sentiment_detector = SentimentDetector() \
    .setInputCols(["lemma", "sentence"]) \
    .setOutputCol("sentiment_score") \
    .setDictionary(resource_path+"sentiment-corpus/default-sentiment-dict.txt", ",")
    
finisher = Finisher() \
    .setInputCols(["sentiment_score"]) \
    .setOutputCols(["sentiment"])

#### 4. Train the pipeline, which is only being trained from external resources, not from the dataset we pass on. The prediction runs on the target dataset

In [15]:
pipeline = Pipeline(stages=[document_assembler, sentence_detector, tokenizer, lemmatizer, sentiment_detector, finisher])
model = pipeline.fit(data)
result = model.transform(data)

#### 5. filter the finisher output, to find the negative sentiment lines

In [16]:
result.filter("sentiment != 'positive'").show(10,False)

+------+---------+-----------------------------------------------------------------------------------------------------------------------------------------+
|itemid|sentiment|text                                                                                                                                     |
+------+---------+-----------------------------------------------------------------------------------------------------------------------------------------+
|102   |negative |  not a cool night.                                                                                                                      |
|135   |negative | #squarespace brighten my bad day! i never win anything...                                                                               |
|145   |negative | &quot;I want you so bad it's driving me  mad&quot;                                                                                      |
|190   |negative | @copicmarker&quot; facebook.com/copic.m